<a href="https://colab.research.google.com/github/mrtlckn/IntroToPytorch/blob/main/101_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import torch

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

 # 1- Load MNIST dataset

In [54]:
# Convert MNIST Image Files into a Tensor of 4-Dimensios(# of images, Height, Widht, Color Channel)
transform = transforms.ToTensor()

In [55]:
# Train Data
train_data = datasets.MNIST(root = '/cnn_data', train=True, download=True, transform=transform)
# Test Data
test_data = datasets.MNIST(root = '/cnn_data', train=False, download=True, transform=transform)



In [56]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [57]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

# 2- Convolutional and Pooling Layers

In [58]:
# Create a small batch size for images... let's say 10
train_loader = DataLoader(train_data, batch_size = 10, shuffle= True)
test_loader = DataLoader(test_data, batch_size = 10, shuffle= False)

In [59]:
# Define Our CNN Model
# Describe CNN layer and what it's doing
# This is just an example in the next chapter we'll build out the actual model

conv1 =  nn.Conv2d(1, 6, 3, 1)
conv2 =  nn.Conv2d(6, 16, 3, 1)


In [60]:
# Grab 1 MNIST Record
for i, (X_train, y_train) in enumerate (train_data):
  break

In [61]:
X_train.shape

torch.Size([1, 28, 28])

In [62]:
x = X_train.view(1,1,28,28) # four dimensions. (1 batch, 1 image and size)

In [63]:
# Perform our first convolution
x = F.relu(conv1(x)) # Rectified Linear Unit for out activation function

In [65]:
# 1 single image, 6 is the filters we asked for, 26x26
x.shape

torch.Size([1, 6, 26, 26])

In [66]:
# pass thru pooling layer
x = F.max_pool2d(x, 2, 2) #kernel of 2 and stride of 2

In [68]:
x.shape # 26 / 2 = 13

torch.Size([1, 6, 13, 13])

In [69]:
# Do our second CNN layer
x = F.relu(conv2(x))

In [71]:
x.shape # Again, we didn't set padding so we lose 2 pixels around the outside of the image

torch.Size([1, 16, 11, 11])

In [72]:
# Pooling layer
x = F.max_pool2d(x,2,2)

In [74]:
x.shape # 11 / 2 = 5.5 but we have to round down, because you can't invent data to round up

torch.Size([1, 16, 5, 5])

# 3- Actual Model

In [75]:
# model Class

class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,6,3,1)
    self.conv2 = nn.Conv2d(6,16,3,1)

    # Fully Connected Layer
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84,10)

  def forward(self,X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X,2,2) #2x2 kernel and stride 2

    # Second Pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X,2,2) #2x2 kernel and stride 2

    # Re-View to flatten it out
    X = X.view(-1, 16,5,5) # Negative one so that we can vary the batch size


    # Fully Connected Layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)

    return F.log_softmax(X, dim=1)


In [78]:
# Create an Instance of our Model

torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [79]:
# Loss Function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) # Smaller the Learning Rate, longer its gonna take to train.